In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [2]:
df = pd.read_pickle("../data/adult.pkl")

In [3]:
df.head()

,age,fnlwgt,education-num,marital-status,relationship,race,sex,capital-gain,capital-loss,hours-per-week,country,salary,employment_type
0,39,77516,13,1,3,0,1,1,0,40,1,1,0
1,50,83311,13,0,2,0,1,0,0,13,1,1,2
2,38,215646,9,1,3,0,1,0,0,40,1,1,1
3,53,234721,7,0,2,3,1,0,0,40,1,1,1
4,28,338409,13,0,1,3,0,0,0,40,0,1,1


# Divide data in training, validation and test dataset
* 50% training data, 20% validation data, 30% test data

In [4]:
from sklearn.model_selection import train_test_split

X= df.drop(['salary'],axis=1)
y= df['salary']

split_size=0.3

#Creation of Train and Test dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=split_size,random_state=22)

#Creation of Train and validation dataset
X_train, X_val, y_train, y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=5)

In [5]:
print ("Train dataset: {0}{1}".format(X_train.shape, y_train.shape))
print ("Validation dataset: {0}{1}".format(X_val.shape, y_val.shape))
print ("Test dataset: {0}{1}".format(X_test.shape, y_test.shape))

Train dataset: (16890, 12)(16890,)
Validation dataset: (4223, 12)(4223,)
Test dataset: (9049, 12)(9049,)


In [6]:
#datas = np.loadtxt('../data/adultpre.csv', delimiter = ',' , dtype = np.float32)
datas = np.array(df, dtype=np.float32)

In [7]:
datas.shape

(30162, 13)

In [8]:
y.shape

(30162,)

In [9]:
y = np.array(y, dtype=np.float32)

In [10]:
y.shape

(30162,)

In [15]:
train_size = int(len(X) * 0.7)

In [16]:
train_size

21113

In [17]:
xtrain, xtest = X[:train_size], X[train_size:]
ytrain, ytest = y[:train_size], y[train_size:]

In [18]:
xtrain.shape

(21113, 12)

In [19]:
X_train.shape

(16890, 12)

In [20]:
y_train.shape

(16890,)

In [21]:
ytrain.shape

(21113,)

In [22]:
X.shape

(30162, 12)

In [23]:
y.shape

(30162,)

In [24]:
# 1. Logistic Regression Hypothesis 생성

# feed_dict에 feed될(to be fed) tensor를 위한 placeholder 설정
X = tf.placeholder(tf.float32, shape=[None, 12])
Y = tf.placeholder(tf.float32, shape=[None])

# tf.Variable(변수)에 random number를 각각 assign
W = tf.Variable(tf.random_normal([12, 1], mean=0.01, stddev=0.01), name='weight')
b = tf.Variable(tf.random_normal([1]), name='bias')

# logistic regression Hypothesis 설정(tf.sigmoid() == 0과 1 사이의 값을 만들기 위한 시그모이드 함수)
hypothesis = tf.sigmoid(tf.matmul(X, W) + b)
hypothesis = tf.clip_by_value(hypothesis, 1e-5, 1-(1e-5))

Instructions for updating:
Colocations handled automatically by placer.


In [25]:
# 2. Cost function 최소화

#cost function(logistic regression에서, W와 b를 찾기 위한 cost)
cost = -tf.reduce_mean(Y * tf.log(hypothesis) + (1 - Y) * tf.log(1 - hypothesis))
train =  tf.train.GradientDescentOptimizer(learning_rate=0.0001).minimize(cost)

# 정확성 측정
# tf.cast()의 첫 번째 파라미터 'if hypothesis > 0.5' then 1'  1 을 float32로 캐스팅(=1.0)
predicted =  tf.cast(hypothesis > 0.5, dtype=tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, Y), dtype=tf.float32))

In [ ]:
# 3. 학습을 통해 실행

with tf.Session() as sess:     
    sess.run(tf.global_variables_initializer())
    feed = {X: X_train, Y: y_train}
    for step in range(100):
        sess.run(train, feed_dict=feed)
        if step % 2000 == 0:
            print(step, sess.run(cost, feed_dict=feed))
       
    h, c, a = sess.run([hypothesis, predicted, accuracy], feed_dict={X: X_train, Y: y_train})
    print("\nHypothesis: ", h,"\nCorrect (Y): ", c, "\nAccuracy: ", a)

0 8.645926
